## 2.10 종합예제
- 스파크 데이터프레임의 스키마 정보를 알아내는 스키마 추론(schema inference)과 파일의 첫 Row를 헤더로 지정하는 옵션을 사용해서 csv파일을 읽음

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Python Spark SQL basic exmaple") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

In [3]:
#.csv("FileStore/tables/data/flight-data/csv/2015-summary.csv")
  flightData2015 = spark \
  .read \
  .option("inferschema", "true") \
  .option("header", "true") \
  .csv("/FileStore/tables/data/flight-data/csv/2015_summary-ebaee.csv")

- 스칼라와 파이썬에서 사용하는 데이터프레임은 불특정 다수의 로우와 컬럼을 갖음. 로우의 수를 알 수 없는 이유는 자연 연산형태의 프랜스포메이션이기 때문이며, 스파크는 각 컬럼의 데이터 타입을 추론하기 위해 적은 양의 데이터를 읽음. DataFrame의 take액션을 호출하면 그제야 결과를 확인할 수 있음

In [5]:
flightData2015.take(3)

Out[10]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

- 정수 데이터 타입인 count컬럼을 기준으로 데이터를 정렬하는 트랜스포메이션을 추가하였음. sort는 트랜스포메이션이기 때문에 데이터에 변화가 일어나지 않지만 explain 메서드를 통해 스파크 쿼리의 실행계획을 확인할 수 있음

In [7]:
# 200개의 셔플 파티션을 생성한 것을 아래에서 확인할 수 있음
flightData2015.sort("count").explain()

== Physical Plan ==
Sort [count#166 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#166 ASC NULLS FIRST, 200), [id=#188]
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#164,ORIGIN_COUNTRY_NAME#165,count#166] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/data/flight-data/csv/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>

- 액션을 호출하면 트랜스포메이션 실행 계획을 시작함. 액션을 시작하기 전 셔플의 출력 파티션 수를 줄임

In [9]:
spark.conf.set("spark.sql.shuffle.partition", "5")
flightData2015.sort("count").take(2)

Out[12]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

### 2.10.1 DataFrame과 SQL
- 스파크 SQL을 사용하면 모든 DataFrame을 테이블이나 뷰(임시 테이블)로 등록한 후 SQL쿼리를 사용할 수 있음. CreateOrReplaceTempView 메서드를 사용함

In [11]:
flightData2015.createOrReplaceTempView("flight_data_2015")

- DataFrame과 SQL 중 간편한 방식으로 트랜스포메이션을 지정할 수 있음. 같은 목적으로 두 종류의 실행 계획을 비교해보면 동일한 기본 실행 계획으로 컴파일된 것을 알 수 있음

In [13]:
query = """
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
"""

sqlWay = spark.sql(query)
dataFrameWay = flightData2015.groupBy('DEST_COUNTRY_NAME').count()

sqlWay.explain()
print("=================")
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#164], functions=[finalmerge_count(merge count#214L) AS count(1)#202L])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#164, 200), [id=#318]
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#164], functions=[partial_count(1) AS count#214L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#164] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/data/flight-data/csv/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
=================
== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#164], functions=[finalmerge_count(merge count#219L) AS count(1)#209L])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#164, 200), [id=#357]
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#164], functions=[partial_count(1) AS count#219L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#164] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/data/flight-data/csv/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

- 특정 위치를 왕래하는 최대 비햇 횟수를 구하기 위해 max함수를 활용. max함수는 필터링을 수행해 단일 로우를 결과로 반환하는 트랜스포메이션임

In [15]:
#SQL
spark.sql("SELECT max(count) FROM flight_data_2015").take(1)

Out[18]: [Row(max(count)=370002)]

In [16]:
#python
from pyspark.sql.functions import max
flightData2015.select(max('count')).take(1)

Out[19]: [Row(max(count)=370002)]

- 상위 5개의 도착 국가를 찾아내는 복잡한 코드

In [18]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
""")

maxSql.show()

+------------------+-----------------+
 DEST_COUNTRY_NAME|destination_total|
+------------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
 Germany| 1468|
Dominican Republic| 1353|
 South Korea| 1048|
 The Bahamas| 955|
 France| 935|
 Colombia| 873|
 Brazil| 853|
 Netherlands| 776|
 China| 772|
 Jamaica| 666|
 Costa Rica| 588|
 El Salvador| 561|
 Panama| 510|
 Cuba| 466|
 Spain| 420|
+------------------+-----------------+
only showing top 20 rows

In [19]:
from pyspark.sql.functions import desc

flightData2015 \
  .groupBy("DEST_COUNTRY_NAME") \
  .sum('count') \
  .withColumnRenamed("sum(count)", "destination_total") \
  .sort(desc("destination_total")) \
  .show()

+------------------+-----------------+
 DEST_COUNTRY_NAME|destination_total|
+------------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
 Germany| 1468|
Dominican Republic| 1353|
 South Korea| 1048|
 The Bahamas| 955|
 France| 935|
 Colombia| 873|
 Brazil| 853|
 Netherlands| 776|
 China| 772|
 Jamaica| 666|
 Costa Rica| 588|
 El Salvador| 561|
 Panama| 510|
 Cuba| 466|
 Spain| 420|
+------------------+-----------------+
only showing top 20 rows

In [20]:
flightData2015 \
  .groupBy("DEST_COUNTRY_NAME") \
  .sum("count") \
  .withColumnRenamed("sum(count)", "destination_total") \
  .sort(desc('destination_total')) \
  .explain()

== Physical Plan ==
Sort [destination_total#373L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(destination_total#373L DESC NULLS LAST, 200), [id=#735]
 +- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#164], functions=[finalmerge_sum(merge sum#377L) AS sum(cast(count#166 as bigint))#369L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#164, 200), [id=#731]
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#164], functions=[partial_sum(cast(count#166 as bigint)) AS sum#377L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#164,count#166] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/data/flight-data/csv/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>

- (1)은 파일을 읽음. 그 다음 (1~2)는 groupBy, sum()을 실행, 마지막으로는 정렬과 limit(5)를 실행하는 순서